In [40]:
"""
POC: LLM-driven parsing of outage emails → Vector DB → Executive Summary → PPT report
"""
import os
import sys
import numpy as np
import re
import json
import uuid
from pathlib import Path
from collections import Counter
from tqdm import tqdm
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from openai import OpenAI
import re
load_dotenv()  # take environment variables from .env file
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
from matplotlib import cm
from pptx import Presentation
from pptx.util import Inches, Pt
from pptx.dml.color import RGBColor
from pptx.enum.text import PP_ALIGN
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from collections import Counter
from tqdm import tqdm


In [41]:
# LLM + Visualization + PPT
import openai
import matplotlib.pyplot as plt
from pptx import Presentation
from pptx.util import Inches, Pt

In [24]:
# -------------------------------
# Read environment variables
# -------------------------------
EMBED_MODEL_NAME = os.getenv("EMBED_MODEL")
COLLECTION_NAME = os.getenv("COLLECTION_NAME") 
DATA_DIR = os.getenv("DATA_DIR")
CHART_DIR_NAME = os.getenv("CHART_DIR")
OUTPUT_JSON_NAME = os.getenv("OUTPUT_JSON")
PPTX_FILE = "partner_outage_summary.pptx"
print(EMBED_MODEL_NAME, CHART_DIR_NAME, COLLECTION_NAME, DATA_DIR)


all-MiniLM-L6-v2 .\charts_llm partner_outages .\logistics_outage_emails


In [25]:
client = OpenAI()


In [56]:
def llm_parse_email(email_text: str):
    """
    Use LLM to extract partner name, date, reason from outage mail.
    Returns a dict.
    """
    prompt = f"""
You are a system that extracts structured data from outage emails.
Given the text below, extract the following fields:
1. partner_name
2. outage_date
3. outage_reason (briefly summarize, e.g., "VPN failure", "Database outage")
4. impact_summary (1-2 lines summarizing what happened)
5. Pick partner name from sender email or signature only

Return a JSON object with keys: partner_name, outage_date, outage_reason, impact_summary.
Do not include any other text. Do not include JSON word at the top

Email:
---
{email_text}
---
If unsure, return "unknown" for the missing fields.
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role":"system","content":"You extract structured outage data from text emails."},
                {"role":"user","content":prompt}
            ],
            temperature=0
        )
        text = response.choices[0].message.content.strip()
        print("LLM response:", text)
        data = json.loads(text)
        return data
    except Exception as e:
        print("Parsing failed:", e)
        return {"partner_name": "unknown", "outage_date": "unknown", "outage_reason": "unknown", "impact_summary": ""}


In [27]:
def llm_generate_summary(partner: str, all_texts: list):
    """
    Generate a concise, bullet-style executive summary for a partner's outages.
    The summary should have 3–4 bullet points focusing ONLY on failure reasons and impacts.
    """
    joined = "\n---\n".join(all_texts[:8])
    prompt = f"""
You are a professional technical report writer.

Given outage notification emails from partner '{partner}', 
create a concise executive summary in 3–4 bullet points ONLY.

Each bullet should:
- Describe the main *failure reason* briefly (2–5 words max)
- Mention the *impact* of that issue succinctly (e.g., "affected data exchange", "caused delays")

Guidelines:
- Do NOT include any greetings or filler text
- Do NOT include next steps or recommendations
- Keep tone professional, crisp, and business-like
- Return only the bullet points in plain text, each starting with a "•"

Messages:
{joined}
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o-mini",
            messages=[
                {"role": "system", "content": "You write concise executive summaries in bullet points."},
                {"role": "user", "content": prompt}
            ],
            temperature=0.3,
            max_tokens=250
        )
        summary = response.choices[0].message.content.strip()
        print(f"Generated summary for {partner}:\n{summary}\n")
        return summary
    except Exception as e:
        print("Summary generation failed:", e)
        return "• Summary unavailable due to API error."


In [29]:
# -------------------------------
# Step 1: Ingest emails
# -------------------------------
def ingest_emails():
    print("Ingesting emails from", DATA_DIR)
    txt_files = sorted(Path(DATA_DIR).glob("*.txt"))
    if not txt_files:
        print("No emails found in", DATA_DIR)
        return
    
    parsed_records = []

    for f in tqdm(txt_files, desc="Parsing and embedding emails"):
        text = f.read_text(encoding="utf8", errors="ignore")
        parsed = llm_parse_email(text)
        parsed["filename"] = f.name
        parsed["raw_text"] = text
        parsed_records.append(parsed)
       
        metadata = {
            "filename": f.name,
            "partner": parsed.get("partner_name", "unknown"),
            "date": parsed.get("outage_date", "unknown"),
            "reason": parsed.get("outage_reason", "unknown"),
            "impact": parsed.get("impact_summary", "")
        }

        
    print("Ingest complete.")
    # Save to JSON
    output_file = Path(OUTPUT_JSON_NAME) / "parsed_output.json"
    with open(output_file, "w", encoding="utf8") as fp:
        json.dump(parsed_records, fp, indent=2, ensure_ascii=False)

    print(f"✅ Parsed data saved to {output_file}")
    return parsed_records

In [48]:
def create_gradient_background(path="gradient_bg.jpg"):
    """Generate a soft blue gradient background for slides."""
    fig, ax = plt.subplots(figsize=(12, 7))
    gradient = np.linspace(0, 1, 256)
    gradient = np.vstack((gradient, gradient))
    ax.imshow(gradient, aspect='auto', cmap=cm.Blues_r, extent=[0, 1, 0, 1])
    ax.axis('off')
    plt.tight_layout(pad=0)
    plt.savefig(path, bbox_inches='tight', pad_inches=0, dpi=150)
    plt.close()
    return path

In [49]:
def generate_ppt(records, logo_path=None):
    """Create a professional 2-slide deck per partner with logo & footer."""
    prs = Presentation()

    # === STYLE SETTINGS ===
    bg_color = RGBColor(230, 240, 255)  # very light blue
    title_color = RGBColor(20, 50, 100)  # dark navy
    text_color = RGBColor(40, 40, 40)    # neutral gray text
    footer_text = "Confidential — Internal Use Only"

    # Helper: set background color
    def set_slide_bg(slide, color):
        fill = slide.background.fill
        fill.solid()
        fill.fore_color.rgb = color

    # Helper: add footer text
    def add_footer(slide):
        footer_box = slide.shapes.add_textbox(Inches(0.5), Inches(6.8), Inches(9), Inches(0.3))
        tf = footer_box.text_frame
        tf.text = footer_text
        p = tf.paragraphs[0]
        p.font.size = Pt(10)
        p.font.color.rgb = RGBColor(100, 100, 100)
        p.alignment = PP_ALIGN.CENTER

    # Helper: add logo if provided
    def add_logo(slide):
        if logo_path and Path(logo_path).exists():
            slide.shapes.add_picture(str(logo_path), Inches(8.5), Inches(0.2), width=Inches(1.5))

    # === COVER SLIDE ===
    cover = prs.slides.add_slide(prs.slide_layouts[6])  # blank layout
    set_slide_bg(cover, bg_color)
    add_logo(cover)

    title_box = cover.shapes.add_textbox(Inches(1), Inches(2.3), Inches(8), Inches(1.5))
    title_tf = title_box.text_frame
    title_tf.text = "Partner Outage Executive Report"
    p = title_tf.paragraphs[0]
    p.font.size = Pt(44)
    p.font.bold = True
    p.alignment = PP_ALIGN.CENTER
    p.font.color.rgb = title_color

    sub_box = cover.shapes.add_textbox(Inches(1), Inches(4), Inches(8), Inches(1))
    sub_tf = sub_box.text_frame
    sub_tf.text = "Generated automatically using LLM-powered analytics"
    sp = sub_tf.paragraphs[0]
    sp.font.size = Pt(20)
    sp.alignment = PP_ALIGN.CENTER
    sp.font.color.rgb = RGBColor(80, 80, 80)

    add_footer(cover)

    # === PREP OUTPUT DIR ===
    chart_dir = Path(CHART_DIR_NAME).expanduser().resolve()
    chart_dir.mkdir(parents=True, exist_ok=True)

    # === GROUP BY PARTNER ===
    partners = {}
    for r in records:
        partners.setdefault(r["partner_name"], []).append(r)

    for partner, recs in tqdm(partners.items(), desc="Creating slides"):
        reasons = [r["outage_reason"] for r in recs]
        counts = Counter(reasons)

        # 1️⃣ EXECUTIVE SUMMARY SLIDE
        slide_summary = prs.slides.add_slide(prs.slide_layouts[6])
        set_slide_bg(slide_summary, bg_color)
        add_logo(slide_summary)
        add_footer(slide_summary)

        # Title
        title_box = slide_summary.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(9), Inches(1))
        title_tf = title_box.text_frame
        title_tf.text = f"{partner} — Executive Summary"
        p = title_tf.paragraphs[0]
        p.font.size = Pt(30)
        p.font.bold = True
        p.alignment = PP_ALIGN.CENTER
        p.font.color.rgb = title_color

        # LLM summary
        texts = [r["raw_text"] for r in recs]
        summary_text = llm_generate_summary(partner, texts)

        # Bullets (aligned)
        body_box = slide_summary.shapes.add_textbox(Inches(1.5), Inches(1.6), Inches(7), Inches(4))
        body_tf = body_box.text_frame
        body_tf.word_wrap = True
        body_tf.clear()

        for line in summary_text.splitlines():
            clean = line.strip("• ").strip("- ").strip()
            if not clean:
                continue
            p = body_tf.add_paragraph()
            p.text = f"• {clean}"
            p.level = 0
            p.font.size = Pt(20)
            p.font.color.rgb = text_color
            p.space_after = Pt(8)
            p.alignment = PP_ALIGN.LEFT

        # 2️⃣ OUTAGE REASON CHART SLIDE
        slide_chart = prs.slides.add_slide(prs.slide_layouts[6])
        set_slide_bg(slide_chart, bg_color)
        add_logo(slide_chart)
        add_footer(slide_chart)

        title_chart = slide_chart.shapes.add_textbox(Inches(0.5), Inches(0.4), Inches(9), Inches(1))
        title_tf = title_chart.text_frame
        title_tf.text = f"{partner} — Outage Reason Distribution"
        p = title_tf.paragraphs[0]
        p.font.size = Pt(28)
        p.font.bold = True
        p.alignment = PP_ALIGN.CENTER
        p.font.color.rgb = title_color

        # Professional bar chart
        plt.figure(figsize=(6, 4))
        colors = plt.cm.Blues(np.linspace(0.4, 0.85, len(counts)))
        plt.bar(counts.keys(), counts.values(),
                color=colors, edgecolor="gray", linewidth=0.8, width=0.35)  # thinner bars

        # Integer y-axis
        plt.gca().yaxis.get_major_locator().set_params(integer=True)

        plt.title("Failure Reason Frequency", fontsize=18, fontweight="bold", pad=12)
        plt.ylabel("Occurrences", fontsize=14)
        plt.xticks(rotation=25, ha="right", fontsize=13)
        plt.yticks(fontsize=13)
        plt.grid(axis="y", linestyle="--", alpha=0.5)
        plt.tight_layout()

        chart_path = chart_dir / f"{partner}_chart.png"
        plt.savefig(chart_path, dpi=200, bbox_inches="tight")
        plt.close()

        slide_chart.shapes.add_picture(str(chart_path), Inches(1), Inches(1.3),
                                       width=Inches(8), height=Inches(4.8))

    # === SAVE PRESENTATION ===
    PPTX_FILE = "partner_outage_summary.pptx"
    prs.save(PPTX_FILE)
    print(f"✅ PPT generated successfully: {PPTX_FILE}")


In [58]:
# -------------------------------
# Run all
# -------------------------------
if __name__ == "__main__":
    records = ingest_emails()
    if records:
        generate_ppt(records)

Ingesting emails from .\logistics_outage_emails


Parsing and embedding emails:   3%|▎         | 1/33 [00:01<00:53,  1.68s/it]

LLM response: {
  "partner_name": "BlueRoute Express",
  "outage_date": "2025-10-10",
  "outage_reason": "Power failure",
  "impact_summary": "Unexpected power failure in the data center affecting B2B communication with some external systems."
}


Parsing and embedding emails:   6%|▌         | 2/33 [00:03<00:52,  1.69s/it]

LLM response: {
  "partner_name": "BlueRoute Express",
  "outage_date": "2025-10-12",
  "outage_reason": "SFTP server down",
  "impact_summary": "The SFTP server is down, affecting B2B communication with some external systems."
}


...

Generated summary for FastLink Shipping:
• SFTP server down - affected B2B communication  
• Firewall blocking outbound connections - affected B2B communication  
• System maintenance window extension - affected B2B communication  
• Database server unresponsive - affected B2B communication



Creating slides:  60%|██████    | 3/5 [00:08<00:05,  2.54s/it]

Generated summary for GlobalFreight Connect:
• Network latency affecting EDI flow - caused delays in B2B communication  
• Unexpected power failure in data center - disrupted B2B communication  
• Network latency impacting EDI flow - caused delays in B2B communication  
• File transmission failure to E2Open - hindered B2B communication

Generated summary for OmniCargo Solutions:
• Unexpected power failure - affected B2B communication  
• Database server unresponsive - affected B2B communication  
• Message queue congestion - affected B2B communication  
• Certificate validation failure - affected B2B communication



Creating slides: 100%|██████████| 5/5 [00:11<00:00,  2.28s/it]

Generated summary for TransEdge Logistics:
• Unexpected power failure - affected B2B communication  
• Firewall blocking connections - disrupted external system interactions

✅ PPT generated successfully: partner_outage_summary.pptx
